In [1]:

from FlavorBuilder.Model2Mass import Model2Mass
import time
from FlavorBuilder.PyDiscrete import Group
import sympy as sp
import numpy as np
import json
import pickle
import flavorpy.modelfitting as mf


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

gap_path = "/users/.../gap-4.11.1/gap" #gap_path
model_file = "../ModelsInPaper/T19xZ4tab5.txt" #model_file


group = Group(57, 1, gap_path, compute_ALL_CGC =  True) #A4 Group


with open(model_file, "rb") as f: #load model from model_fule
    model_json = json.load(f)


#Grouping the particles in th models
particle_list_grouping2 =[["leftLeptons", "L"], ["rightLeptons", "E"], ["rightNeutrinos", "N"],  ["flavons", "phi"], ["higgsup", "Hu"], ["higgsdown", "Hd"]]

#Operators from the Lagrangian
allOperators2 =  [
                        [["rightNeutrinos", "rightNeutrinos"], 1.0],
                        [["flavons", "rightNeutrinos", "rightNeutrinos"], 0.1],
                        [["flavons", "flavons", "rightNeutrinos", "rightNeutrinos"], 0.01],
                        [["rightNeutrinos", "leftLeptons", "higgsup"], 1],
                        [["rightNeutrinos", "leftLeptons", "higgsup", "flavons"], 0.1],
                        [["leftLeptons", "rightLeptons", "higgsdown"], 1],
                        [["leftLeptons", "rightLeptons", "higgsdown", "flavons"], 0.1]
                    ]


#Load Model
model = Model2Mass(group, model = model_json, particle_list_grouping = particle_list_grouping2)

#Compute lagrangian and params
lag, params = model.get_lagrangian(allOperators = allOperators2)

#Compute mass matrices
mC =  model.get_mass_matrix(lag ,params, ["leftLeptons", "L"], ["rightLeptons", "E"], higgs_fields = ["higgsdown", "Hd"])
mD =  model.get_mass_matrix(lag ,params, ["leftLeptons", "L"], ["rightNeutrinos", "N"], higgs_fields = ["higgsup", "Hu"])
mM =  model.get_mass_matrix(lag ,params, ["rightNeutrinos", "N"], ["rightNeutrinos", "N"])


#mass matrices ranks
mCrank  = model.get_rank_mass_matrix(mC,params)
mDrank  = model.get_rank_mass_matrix(mD,params)
mMrank  = model.get_rank_mass_matrix(mM,params)


#alpha params
effparameters = model.ParametersInAMatrix(mM) + model.ParametersInAMatrix(mD)
effparametersC_alpha =  model.ParametersInAMatrix(mC)

#effetive flavons
eff_flavons, flavons_Not_present = model.count_eff_flavons(lag)





In [3]:
effparams = model.ParametersInAMatrix(mC) + model.ParametersInAMatrix(mD) + model.ParametersInAMatrix(mM)
effparameters_string = [str(symbol) for symbol in effparams]

Me_func = sp.lambdify(effparams, mC, modules="numpy")

MD_func = sp.lambdify(effparams, mD, modules="numpy")

MM_func = sp.lambdify(effparams, mM, modules="numpy")

def Me(params_temp): # Charged lepton mass matrix
    listOfvals = []
    for string in effparameters_string:
        globals()[string+'_val'] = params_temp[string]  
        listOfvals.append(globals()[string+'_val'])
    Mematrix = Me_func(*listOfvals)
    
    return (0.1/np.sqrt(3))*np.array(Mematrix).astype(np.complex128)  


def Mn(params_temp): #Neutrino mass matrix after integrating out right-handed neutrinos
    listOfvals = []
    for string in effparameters_string:
        globals()[string+'_val'] = params_temp[string] 
        listOfvals.append(globals()[string+'_val'])
    
    MDmatrix = MD_func(*listOfvals)
    MMmatrix = MM_func(*listOfvals)

    return  np.matmul(np.matmul(np.array(MDmatrix).astype(np.complex128).T,np.linalg.inv(np.array(MMmatrix).astype(np.complex128) )),np.array(MDmatrix).astype(np.complex128) )


ParamSpace = mf.ParameterSpace() # This creates a parameter space for flavorpy to fit

# Strings for the parameters in the mass matrices
effparameters_string = [str(symbol) for symbol in model.ParametersInAMatrix(mC) + model.ParametersInAMatrix(mD) + model.ParametersInAMatrix(mM)]

for string in effparameters_string:
    ParamSpace.add_dim(name = string)
    
MyModel = mf.FlavorModel(mass_matrix_e=Me, 
                         mass_matrix_n=Mn, 
                         parameterspace=ParamSpace, 
                         ordering='NO', #Normal Ordering
                         fitted_observables=[ 'me/mu', 'mu/mt', 's12^2', 's13^2', 's23^2', 'd/pi', 'r']) 
# Consider only dimensionless observables




In [4]:
mymodel_dict_from_model_file = model_json["param_dict"].copy()
MyModel.print_chisq(mymodel_dict_from_model_file)

'me/mu': 0.00480000000225334,   chisq: 1.2693861262001838e-16
'mu/mt': 0.056500320287136085,   chisq: 5.065869113125379e-09
's12^2': 0.3409292712387581,   chisq: 7.417913754922212
's13^2': 0.02228009897133566,   chisq: 9.518476730072112e-08
's23^2': 0.46460587595428554,   chisq: 0.37351092759543664
'd/pi': 1.6059401649346732,   chisq: 1.809575397264727
'r': 0.029583046581894153,   chisq: 3.926225859512587e-06
Total chi-square: 9.601004106258873


In [5]:
model_json["total_chisq"]

9.601004106258785